In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER']="PCI_BUS_ID"
os.environ['CUDA_VISIBLEЕ_DEVICES']="0"
from tensorflow.keras import Model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
import tensorflow as tf
# GPU санах ойн өсөлтийг зөвхөн шаардлагатай зүйлээр хязгаарлаарай
# gpu санах ойн өсөлтийг зөвхөн хэрэгцээтэй зүйлээр хязгаарлах
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # inviald 
    print('Failed')

In [ ]:
# datasets link https://www.kaggle.com/datasets/crowww/a-large-scale-fish-dataset?resource=download
train_path = r"C:\Users\mbsav\Documents\hicheel\hiimel oyun\biy-daalt\Fish\Fish_Dataset\Fish_Dataset"
valid_path = r"C:\Users\mbsav\Documents\hicheel\hiimel oyun\biy-daalt\Fish\Fish_Dataset\Validation"
train_generator = ImageDataGenerator(rotation_range=15, width_shift_range=.1, height_shift_range=.1, brightness_range=(0, .2)).flow_from_directory(train_path, target_size=(590,445), batch_size=30)

valid_generator = ImageDataGenerator(rotation_range=15, width_shift_range=.1, height_shift_range=.1, brightness_range=(0, .2)).flow_from_directory(valid_path, target_size=(590,445), batch_size=30)


In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
preds = Dense(9, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=preds)


In [ ]:
model.summary()

In [ ]:
for layers in model.layers[:5]:
    layers.trainable = False

In [ ]:
model.summary()

In [ ]:
epochs = 20
optimizer = Adam(learning_rate=.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(train_generator, validation_data=valid_generator, epochs=epochs)
